En el siguiente ejemplo vamos a implementar una CNN que permita clasificacion multiclase.

El primer paso es crear 2 carpetas que son: la carpeta de entrenamiento y la carpeta de pruebas; luego dentro de cada una de ellas se deben ubicar N carpetas que denotan cada una de las clases, y dentro de cada una de estas carpetas deben  contener las respectivas imagenes correspondientes a la clase. 

![folder_data](img/folder_data.jpeg)

Keras tiene una clase llamada ImageDataGenerator que permite hacer aumentos en tiempo real de las imagenes.En este ejmplo vamos a utilizar el metodo flow_from_directory().

Es muy importante resaltar que el metodo flow_from_directory() espera una estructura similar a la que se presentan en la figura anterior para su correcto funcionamiento.

Los nombres de las carpetas para las clases son importantes, nombrarlas (o renombrarlas) con los nombres de las etiquetas respectivas para que sea más fácil para usted más tarde.

El primer paso es crear una instancia de la clase ImageDataGenerator con algunas configuraciones basicas, como rotacion de 40 grados de la imagen, normalizacion de la imagen, acercamiento aleatorio de la imagen entre otros opciones disponibles para su consulta en el siguiente enlace:https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [27]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras import backend as K

In [28]:
img_width, img_height = 150, 150

In [29]:
nb_train_samples = 3000
nb_validation_samples = 320
epochs = 50
batch_size = 400

In [30]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [31]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [32]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [33]:
ENTRENAMIENTO_DIR = 'data/data/entrenamiento/'
TEST_DIR = 'data/data/pruebas/'

A continuacion se presenta la siguiente instruccion con los parametros comunmente utilizados para flow_from_directory:

In [8]:
train_generator = train_datagen.flow_from_directory(
    directory=ENTRENAMIENTO_DIR,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2002 images belonging to 2 classes.


- directory: ruta donde se encuentran las imagenes.
- target_size: las imagenes de entrada sera redimenzionadas al tamaño que determine.
- color_mode: si la imagen es en escala de grises colocar "grayscale", si es a color "rgb"
- batch_size: numero de imagenes que se procesaran por el generador de lotes.
- class_mode: si es un problema binario configure "binary", si es multi clases configure "categorical"
- shuffle: configure True, si desea barajar las imagenes, de lo contrario configure False.
- seed: Semilla aleatoria para aplicar un aumento de imagen aleatorio y barajar el orden de la imagen.

Igual paso para las imagenes de pruebas:

In [9]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [10]:
validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 400 images belonging to 2 classes.


En el caso anterior se configuro class_mode como None, porque solo requerimos que nos retorne las imagenes. En el caso de shuffle establezca esta opción en False, ya que necesita producir las imágenes en "orden", para predecir las salidas y hacerlas coincidir con sus ids o nombres de archivo únicos.

Creamos la CNN

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxP

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
125/125 [==============================] - 65s 520ms/step - loss: 0.7058 - acc: 0.5235 - val_loss: 0.6732 - val_acc: 0.6025
Epoch 2/50
125/125 [==============================] - 62s 496ms/step - loss: 0.6660 - acc: 0.6015 - val_loss: 0.6538 - val_acc: 0.6175
Epoch 3/50
125/125 [==============================] - 63s 501ms/step - loss: 0.6342 - acc: 0.6645 - val_loss: 0.5978 - val_acc: 0.6700
Epoch 4/50
125/125 [==============================] - 64s 509ms/step - loss: 0.6167 - acc: 0.6700 - val_loss: 0.6184 - val_acc: 0.6625
Epoch 5/50
125/125 [==============================] - 65s 523ms/step - loss: 0.5853 - acc: 0.7125 - val_loss: 0.5588 - val_acc: 0.7275
Epoch 6/50
125/125 [==============================] - 68s 541ms/step - loss: 0.5637 - acc: 0.7095 - val_loss: 0.5925 - val_acc: 0.7175
Epoch 7/50
125/125 [==============================] - 67s 533ms/step - loss: 0.5608 - acc: 0.7225 - val_loss: 0.5965 - val_acc: 0.6825
Epoch 8

En el siguiente ejemplo entrenaremos la CNN para clasificacion multiclase, para lo cual utilizaremos el siguiente dataset disponible en Kaggle: https://www.kaggle.com/jutrera/stanford-car-dataset-by-classes-folder/downloads/stanford-car-dataset-by-classes-folder.zip/2.<br>
Las modificaciones que debemos realizar es configurar la perdidad como categorical_crossentropy y el parametro class_mode del metodo flow_from_directory en categorical.


Primer paso definimos las rutas de la carpeta donde se encuentran ubicadas las imagenes, es importante respetar la estructura de carpetas para que el metodo flow_from_directory puede funcionar correctamente.

In [34]:
ENTRENAMIENTO_DIR = 'data/flowers/train/'
TEST_DIR = 'data/flowers/test/'

In [35]:
train_generator = train_datagen.flow_from_directory(
    directory=ENTRENAMIENTO_DIR,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3523 images belonging to 5 classes.


In [36]:
validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 800 images belonging to 5 classes.


In [43]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(96, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))

#model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_31 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
activation_32 (Activation)   (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 34, 34, 96)        55392     
__________

In [44]:
model.fit_generator(
    train_generator,
    steps_per_epoch= 3523 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=800  // batch_size)

Epoch 1/50
8/8 [==============================] - 174s 22s/step - loss: 1.5649 - acc: 0.2778 - val_loss: 1.4159 - val_acc: 0.3850
Epoch 2/50
8/8 [==============================] - 157s 20s/step - loss: 1.4179 - acc: 0.3608 - val_loss: 1.3193 - val_acc: 0.4675
Epoch 3/50
8/8 [==============================] - 137s 17s/step - loss: 1.2542 - acc: 0.4889 - val_loss: 1.1315 - val_acc: 0.5350
Epoch 4/50
8/8 [==============================] - 140s 17s/step - loss: 1.1130 - acc: 0.5438 - val_loss: 1.1721 - val_acc: 0.5375
Epoch 5/50
8/8 [==============================] - 145s 18s/step - loss: 1.0590 - acc: 0.5705 - val_loss: 1.0137 - val_acc: 0.6062
Epoch 6/50
8/8 [==============================] - 127s 16s/step - loss: 1.0120 - acc: 0.5887 - val_loss: 0.9844 - val_acc: 0.5925
Epoch 7/50
8/8 [==============================] - 153s 19s/step - loss: 0.9589 - acc: 0.6314 - val_loss: 0.9513 - val_acc: 0.6487
Epoch 8/50
8/8 [==============================] - 143s 18s/step - loss: 0.8867 - acc: 0.65